In [1]:
from graphnet.models.gnn  import  DynEdge

graphnet [MainProcess] WARNING  2026-01-10 04:54:54 - has_jammy_flows_package - `jammy_flows` not available. Normalizing Flow functionality is missing.


In [2]:
# after pmt/dom posn and homophily decision, you may need to inherit from this and write your own class

In [3]:
backbone = DynEdge(nb_inputs=8, # Number of input features on each node. affects MLP input in first layer and global parameters
                      nb_neighbours=8, # Number of neighbours to used in the k-nearest neighbour clustering which is performed after each (dynamical) edge convolution. same in all DynEdgeConv layers
                      # features_subset : The subset of latent features on each node that are used as metric dimensions when performing the k-nearest neighbours clustering. Defaults to [0,1,2]. same in all DynEdgeConv layers. 
                      # dynedge_layer_sizes : default = [(128,256), (336,256), (336,256), (336,256)]. Defn: The layer sizes, or latent feature dimenions, used in the `DynEdgeConv` layer. Each entry in `dynedge_layer_sizes` corresponds to a single `DynEdgeConv` layer; the integers in the corresponding tuple corresponds to the layer sizes in the multi-layer perceptron (MLP) that is applied within each `DynEdgeConv` layer. That is, a list of size-two tuples means that all `DynEdgeConv` layers contain a two-layer MLP. (128,256) means MLP: ... -> 128 -> 256 
                      # post_processing_layer_sizes : default = [336, 256]. Defn: Hidden layer sizes in the MLP following the skip-concatenation of the outputs of each `DynEdgeConv` layer. 
                      # readout_layer_sizes : default = [128, ]. defn: Hidden layer sizes in the MLP following the post-processing _and_ optional global pooling. As this is the last layer(s) in the model, the last layer in the read-out yields the output of the `DynEdge` model.
                      global_pooling_schemes = ["min","max","mean","sum"],
                      add_global_variables_after_pooling = True,  # if you set this to False, then the global parameters are added to the each pulse feature vector (each pulse has the same addition). Try False later 
                      activation_layer = "relu", # try gelu later
                      add_norm_layer = False, # try True later
                      skip_readout = False                      
                     )

# self._nb_global_variables = 5 + nb_inputs  : global parameters = mean of each input + homophily's + 1 (number of pulse)
# write your own DynEdge class and change the 5 if necessary. 

# by default, sum aggregation is used in DynEdge


graphnet [MainProcess] INFO     2026-01-10 04:54:54 - DynEdge.__init__ - Writing log to logs/graphnet_20260110-045454.log


In [4]:
backbone

DynEdge(
  DynEdge(
  {
      'arguments': {
          'nb_inputs': 8,
          'nb_neighbours': 8,
          'features_subset': None,
          'dynedge_layer_sizes': None,
          'post_processing_layer_sizes': None,
          'readout_layer_sizes': None,
          'global_pooling_schemes': ['min', 'max', 'mean', 'sum'],
          'add_global_variables_after_pooling': True,
          'activation_layer': 'relu',
          'add_norm_layer': False,
          'skip_readout': False,
      },
  })
  (_activation): ReLU()
  (_conv_layers): ModuleList(
    (0): DynEdgeConv(nn=Sequential(
      (0): Linear(in_features=16, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=256, bias=True)
      (3): ReLU()
    ))
    (1-3): 3 x DynEdgeConv(nn=Sequential(
      (0): Linear(in_features=512, out_features=336, bias=True)
      (1): ReLU()
      (2): Linear(in_features=336, out_features=256, bias=True)
      (3): ReLU()
    ))
  )
  (_post_processing):

In [5]:
backbone._conv_layers

ModuleList(
  (0): DynEdgeConv(nn=Sequential(
    (0): Linear(in_features=16, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): ReLU()
  ))
  (1-3): 3 x DynEdgeConv(nn=Sequential(
    (0): Linear(in_features=512, out_features=336, bias=True)
    (1): ReLU()
    (2): Linear(in_features=336, out_features=256, bias=True)
    (3): ReLU()
  ))
)

In [6]:
backbone._post_processing

Sequential(
  (0): Linear(in_features=1032, out_features=336, bias=True)
  (1): ReLU()
  (2): Linear(in_features=336, out_features=256, bias=True)
  (3): ReLU()
)

In [7]:
backbone._readout

Sequential(
  (0): Linear(in_features=1037, out_features=128, bias=True)
  (1): ReLU()
)

In [8]:
# forward expects variable named "data" that has the parts:  data.x, data.edge_index, data.batch
# calculate it using knn first.

In [9]:
# this backbone's output has 128 dimensions. join it to a head (MLP/Linear)